In [145]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import stats
pd.options.display.max_columns = None


In [146]:
df_raw=pd.read_csv("../../flight_data_raw.csv")

In [147]:
df_raw.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,V26
0,2018,4,26,4,UA,DTW,EWR,910,1107.0,117.0,1.0,1053,1253.0,120.0,1.0,0,NaN,0,73.0,488,33.0,0.0,3.0,0.0,84.0,NaN
1,2018,4,26,4,UA,EWR,RDU,1400,1349.0,-11.0,0.0,1541,1521.0,-20.0,0.0,0,NaN,0,74.0,416,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,4,26,4,UA,SFO,EUG,1059,1115.0,16.0,1.0,1232,1242.0,10.0,0.0,0,NaN,0,61.0,451,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,4,26,4,UA,TPA,IAD,1858,1853.0,-5.0,0.0,2111,2058.0,-13.0,0.0,0,NaN,0,108.0,811,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,4,26,4,UA,SFO,BOS,1630,1625.0,-5.0,0.0,118,114.0,-4.0,0.0,0,NaN,0,316.0,2704,NaN,NaN,NaN,NaN,NaN,NaN


Remove extreme outliers

In [148]:
df=df_raw[(np.abs(df_raw.ARR_DELAY-df_raw.ARR_DELAY.mean()) <= (3*df_raw.ARR_DELAY.std())) | (df_raw["CANCELLED"]==1)]
# keep only the ones that are within +3 to -3 standard deviations in the column 'ARR_DELAY'.


In [149]:
count=df.groupby(["ORIGIN","DEST"]).size()
count


ORIGIN  DEST
ABE     ATL      942
        CLT      942
        DTW      887
        FLL       67
        MYR       86
                ... 
YAK     CDV      355
        JNU      360
YNG     PIE        1
        SFB        1
YUM     PHX     1371
Length: 6327, dtype: int64

In [150]:
df[df["CANCELLED"]==1]

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DEL15,CANCELLED,CANCELLATION_CODE,DIVERTED,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,V26
477,2018,4,26,4,UA,DEN,KOA,1145,1143.0,-2.0,0.0,1522,NaN,NaN,NaN,1,A,0,NaN,3329,NaN,NaN,NaN,NaN,NaN,NaN
500,2018,4,26,4,UA,KOA,DEN,2003,NaN,NaN,NaN,655,NaN,NaN,NaN,1,A,0,NaN,3329,NaN,NaN,NaN,NaN,NaN,NaN
1992,2018,4,25,3,UA,IAD,SFO,815,NaN,NaN,NaN,1112,NaN,NaN,NaN,1,A,0,NaN,2419,NaN,NaN,NaN,NaN,NaN,NaN
2522,2018,4,25,3,UA,EWR,BOS,1601,NaN,NaN,NaN,1715,NaN,NaN,NaN,1,C,0,NaN,200,NaN,NaN,NaN,NaN,NaN,NaN
2933,2018,4,25,3,UA,BOS,EWR,1801,NaN,NaN,NaN,1929,NaN,NaN,NaN,1,C,0,NaN,200,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7210656,2018,9,13,4,UA,CHS,ORD,1850,NaN,NaN,NaN,2016,NaN,NaN,NaN,1,B,0,NaN,760,NaN,NaN,NaN,NaN,NaN,NaN
7210818,2018,9,13,4,UA,ORD,ORF,1800,NaN,NaN,NaN,2108,NaN,NaN,NaN,1,B,0,NaN,717,NaN,NaN,NaN,NaN,NaN,NaN
7210879,2018,9,13,4,UA,CHS,EWR,1425,NaN,NaN,NaN,1627,NaN,NaN,NaN,1,B,0,NaN,628,NaN,NaN,NaN,NaN,NaN,NaN
7210896,2018,9,13,4,UA,CHS,ORD,1535,NaN,NaN,NaN,1701,NaN,NaN,NaN,1,B,0,NaN,760,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
#this works to get airport list
#df2 = df[["ORIGIN","DEST"]].groupby('ORIGIN').aggregate(lambda tdf: tdf.unique().tolist())

In [152]:
df.shape

(7074679, 26)

Atlanta has 166 Airports they fly to!

Create the final output csv:

Number of flights

In [153]:
x = df.groupby(["ORIGIN","DEST"]).size()

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r = x.reindex(idx).dropna()
r=r.reset_index()
          


Average delay

In [154]:

y = df.groupby(["ORIGIN","DEST"]).mean()["ARR_DELAY"]

idx = pd.MultiIndex.from_product((df.ORIGIN.unique(), df.DEST.unique()))

r_2 = y.reindex(idx).dropna()

r_2=r_2.reset_index() 
#r_2.groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x["ARR_DELAY"])))\

Causes of delay as percentage

In [155]:

z = df.groupby(["ORIGIN","DEST"]).sum()[["CANCELLED","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]]

r_3 = z.reindex(idx).dropna()

r_3=r_3.reset_index() 
#r_2.groupby('level_0').apply(lambda x: dict(zip(x['level_1'],x["ARR_DELAY"])))\
r_3

,level_0,level_1,CANCELLED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,DTW,EWR,160.0,5071.0,2864.0,40226.0,4.0,8924.0
1,DTW,RDU,5.0,2517.0,54.0,2315.0,0.0,2205.0
2,DTW,IAD,36.0,4040.0,726.0,3612.0,0.0,6902.0
3,DTW,BOS,50.0,7333.0,704.0,16380.0,0.0,8233.0
4,DTW,SAT,2.0,2842.0,76.0,1257.0,0.0,1015.0
...,...,...,...,...,...,...,...,...
6322,EAR,DEN,1.0,14.0,169.0,198.0,0.0,69.0
6323,ART,PHL,1.0,153.0,15.0,13.0,9.0,81.0
6324,IFP,PHX,0.0,15.0,0.0,303.0,0.0,91.0
6325,YNG,SFB,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
r_join= r.merge(r_3, how="left", on=["level_0","level_1"])
r_join= r_join.merge(r_2, how="left", on=["level_0","level_1"])
r_join.rename(columns= {0: "flight_volume","level_0": "origin","level_1": "destination","ARR_DELAY":"arr_delay"}, inplace=True)


In [157]:
r_join["VOLUME_RANK"]=r_join["FLIGHT_VOLUME"].rank(method='max',ascending=False)

In [158]:
r_join.to_csv("flights.csv")

,level_0,level_1,FLIGHT_VOLUME,CANCELLED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,ARR_DELAY,VOLUME_RANK
0,DTW,EWR,3713.0,160.0,5071.0,2864.0,40226.0,4.0,8924.0,8.936392,336.0
1,DTW,RDU,1302.0,5.0,2517.0,54.0,2315.0,0.0,2205.0,-4.052429,1838.0
2,DTW,IAD,1801.0,36.0,4040.0,726.0,3612.0,0.0,6902.0,4.009065,1248.0
3,DTW,BOS,3672.0,50.0,7333.0,704.0,16380.0,0.0,8233.0,0.070955,348.0
4,DTW,SAT,836.0,2.0,2842.0,76.0,1257.0,0.0,1015.0,-6.158273,2734.0
...,...,...,...,...,...,...,...,...,...,...,...
6322,EAR,DEN,197.0,1.0,14.0,169.0,198.0,0.0,69.0,-17.989796,4584.0
6323,ART,PHL,23.0,1.0,153.0,15.0,13.0,9.0,81.0,3.727273,5915.0
6324,IFP,PHX,45.0,0.0,15.0,0.0,303.0,0.0,91.0,6.933333,5668.0
6325,YNG,SFB,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,6327.0


In [159]:

target=r_join.groupby('ORIGIN')[['DESTINATION',"FLIGHT_VOLUME","ARR_DELAY","CANCELLED","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY","VOLUME_RANK"]].apply(lambda x: x.set_index('level_1').to_dict(orient='index'))
target

level_0
ABE    {'FLL': {'FLIGHT_VOLUME': 67.0, 'ARR_DELAY': 2...
ABI    {'DFW': {'FLIGHT_VOLUME': 1977.0, 'ARR_DELAY':...
ABQ    {'SAT': {'FLIGHT_VOLUME': 18.0, 'ARR_DELAY': 2...
ABR    {'MSP': {'FLIGHT_VOLUME': 725.0, 'ARR_DELAY': ...
ABY    {'ATL': {'FLIGHT_VOLUME': 965.0, 'ARR_DELAY': ...
                             ...                        
WYS    {'SLC': {'FLIGHT_VOLUME': 238.0, 'ARR_DELAY': ...
XNA    {'EWR': {'FLIGHT_VOLUME': 305.0, 'ARR_DELAY': ...
YAK    {'CDV': {'FLIGHT_VOLUME': 355.0, 'ARR_DELAY': ...
YNG    {'SFB': {'FLIGHT_VOLUME': 1.0, 'ARR_DELAY': 9....
YUM    {'PHX': {'FLIGHT_VOLUME': 1371.0, 'ARR_DELAY':...
Length: 358, dtype: object

In [160]:
r

,level_0,level_1,0
0,DTW,EWR,3713.0
1,DTW,RDU,1302.0
2,DTW,IAD,1801.0
3,DTW,BOS,3672.0
4,DTW,SAT,836.0
...,...,...,...
6322,EAR,DEN,197.0
6323,ART,PHL,23.0
6324,IFP,PHX,45.0
6325,YNG,SFB,1.0


In [161]:
target.to_csv("airports_serviced_data.csv")